# Task3 Building Summary Tables

In this task,I will build a single SQLite database via Python (in a .db file) containing three tables:
1.	Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items .
2.	Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items.
3.	Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.

**Notes: Task3 is based on the task2, which means I used the data of my own sample owners to do this summary task. If I made a wrong understand, please let me know.**

**Notes: Final Sample files are uploaded into the GBQ, access [here](https://console.cloud.google.com/bigquery?project=hong-wedge&p=hong-wedge&d=transactions&t=sample_owners_records&page=table)!**

In [1]:
import os
import io
import csv
from zipfile import ZipFile

from google.cloud import bigquery
from google.oauth2 import service_account

## Connects to GBQ instance.


In [54]:
# GBQ Setting
service_path = "./"
service_file = 'Hong-Wedge-8a5b036bb32c.json' 
gbq_proj_id = 'hong-wedge' 

private_key =service_path + service_file
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

## Part1: Generate the summary data files
1. Generate summary tables via queries in Google Big Query
2. Export the text files, and store them locally on the machine. 

Actually, this part is a little similar with the task2

Reference: [Date Process in GBQ](https://cloud.google.com/bigquery/docs/reference/standard-sql/datetime_functions#datetime)

In [35]:
def generate_data(query,fields,file_name):
    ## Define the query_job to execute
    query_job = client.query(
    query,
    location="US",)
    
    ## execute and print the data
    with open(file_name, "w",encoding="utf-8") as text_file:
        text_file.write(",".join(fields)+"\n")
        for idx, row in enumerate(query_job) :
            record=[]
            for i in range(0,len(row)):
                record.append(str(row[i]))
            output=",".join(record)
            text_file.write(output+"\n")
    print("Done!")
    

#### Sales by date by hour 
By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items .

In [41]:
query = (
    "SELECT EXTRACT(DATE FROM datetime) as day,EXTRACT(HOUR FROM datetime) as hour,"
    "       sum(total) as spends,"
    "       count(distinct(date(datetime) || register_no ||emp_no || trans_no)) as transactions,"
    "       sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as items "
    "FROM `hong-wedge.transactions.sample_owners_records` "
    "WHERE card_no!=3 and trans_status in('',' ') "
    "GROUP BY Day, Hour "
    "ORDER BY Day, Hour "
)

fields=["day","hour","spends","transactions","items"]

generate_data(query,fields,"sales_by_date_by_hour.csv")

Done!


#### Sales by owner by year by month
A file that has the following columns: card_no, year, month, sales, transactions, and items.

In [42]:
# SELECT card_no,EXTRACT(YEAR FROM datetime) as year,EXTRACT(MONTH FROM datetime) as month,sum(total) as sales, count(trans_no) as NumberOfTransactions,count(trans_status) as NumberOfItems
# FROM `hong-wedge.transactions.transArchive_201104` 
# WHERE card_no!=3 and trans_status in(""," ")
# GROUP BY card_no, year, month
# ORDER BY card_no, year, month


query = (
    "SELECT card_no,EXTRACT(YEAR FROM datetime) as year,EXTRACT(MONTH FROM datetime) as month,"
    "       sum(total) as spends,"
    "       count(distinct(date(datetime) || register_no ||emp_no || trans_no)) as transactions,"
    "       sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as items "
    "FROM `hong-wedge.transactions.sample_owners_records` "
    "WHERE trans_status in('',' ') "
    "GROUP BY card_no, year, month "
    "ORDER BY card_no, year, month "
)

fields=["card_no","year","month","sales","transactions","items"]

generate_data(query,fields,"sales_by_owner_by_year_by_month.csv")

Done!


#### Sales by product description by year by month
A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.

In [43]:
# SELECT upc,description,department,dept_name as department_name,EXTRACT(YEAR FROM datetime) as year,EXTRACT(MONTH FROM datetime) as month,sum(total) as sales, count(trans_no) as transactions,count(trans_status) as items
# FROM `hong-wedge.transactions.transArchive_201104` join ` department_lookup` on department 
# WHERE card_no!=3 and trans_status in(""," ")
# GROUP BY upc,description,department,department_name year, month
# ORDER BY upc,description,department,department_name year, month

query = (
    "SELECT upc,description,`hong-wedge.transactions.sample_owners_records`.department,dept_name as department_name,"
    "       EXTRACT(YEAR FROM datetime) as year,EXTRACT(MONTH FROM datetime) as month,"
    "       sum(total) as spends,"
    "       count(distinct(date(datetime) || register_no ||emp_no || trans_no)) as transactions,"
    "       sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as items "
    "FROM `hong-wedge.transactions.sample_owners_records` join `hong-wedge.transactions.department_lookup` "
    "on `hong-wedge.transactions.sample_owners_records`.department = `hong-wedge.transactions.department_lookup`.department "
    "WHERE trans_status in('',' ') "
    "GROUP BY upc,description,department,department_name, year, month "
    "ORDER BY upc,description,department,department_name, year, month "
)

fields=["upc","description","department","department_name","year","month","sales","transactions","items"]

generate_data(query,fields,"sales_by_product_by_year_by_month.csv")

Done!


## Part2: Create Database
1. Creates a single SQLite database 
2. Creates the tables
3. Fills those tables. 

In [46]:
import sqlite3
db = sqlite3.connect("wedge_summary.db")
cur = db.cursor()

In [50]:
#fields=["day","hour","spends","transactions","items"]

cur.execute('''DROP TABLE IF EXISTS sales_by_date_by_hour''')
cur.execute('''CREATE TABLE sales_by_date_by_hour (
    day TIMESTAMP, 
    Hour INTEGER, 
    spends REAL,
    transactions REAL,
    items REAL)''')

with open("sales_by_date_by_hour.csv",'r', encoding="utf-8") as ifile :
    next(ifile)
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO sales_by_date_by_hour(day,hour,spends,transactions,items)
            VALUES (?,?,?,?,?)''', line)        
db.commit()

test = cur.execute('''SELECT * FROM sales_by_date_by_hour LIMIT 10''')
print(list(test))

[('2010-01-01', 9, 41.99, 1.0, 5.0), ('2010-01-01', 10, 4.77, 1.0, 3.0), ('2010-01-01', 11, 281.79000000000025, 3.0, 83.0), ('2010-01-01', 13, 26.07, 2.0, 8.0), ('2010-01-01', 14, 34.4, 1.0, 9.0), ('2010-01-01', 15, 16.75, 1.0, 3.0), ('2010-01-01', 17, 17.15, 3.0, 7.0), ('2010-01-01', 19, 3.37, 1.0, 1.0), ('2010-01-01', 20, 52.400000000000006, 1.0, 17.0), ('2010-01-01', 21, 19.86, 2.0, 6.0)]


In [51]:
#fields=["card_no","year","month","sales","transactions","items"]
cur.execute('''DROP TABLE IF EXISTS sales_by_owner_by_year_by_month''')
cur.execute('''CREATE TABLE sales_by_owner_by_year_by_month (
    card_no INTEGER, 
    year INTEGER, 
    month INTEGER,
    sales REAL,
    transactions REAL,
    items REAL)''')

with open("sales_by_owner_by_year_by_month.csv",'r', encoding="utf-8") as ifile :
    next(ifile)
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO sales_by_owner_by_year_by_month(card_no,year,month,sales,transactions,items)
            VALUES (?,?,?,?,?,?)''', line)        
db.commit()

test = cur.execute('''SELECT * FROM sales_by_owner_by_year_by_month LIMIT 10''')
print(list(test))

[(10037, 2010, 1, 133.57999999999998, 5.0, 26.0), (10037, 2010, 2, 166.99999999999994, 5.0, 44.0), (10037, 2010, 3, 178.74999999999994, 8.0, 42.0), (10037, 2010, 4, 117.63, 7.0, 35.0), (10037, 2010, 5, 37.809999999999995, 3.0, 11.0), (10037, 2010, 6, 200.53, 11.0, 58.0), (10037, 2010, 7, 174.33999999999995, 7.0, 45.0), (10037, 2010, 8, 47.170000000000016, 8.0, 76.0), (10037, 2010, 9, -6.217248937900877e-15, 3.0, 34.0), (10037, 2010, 10, 2.1760371282653068e-14, 4.0, 44.0)]


In [52]:
#fields=["upc","description","department","department_name","year","month","sales","transactions","items"]
cur.execute('''DROP TABLE IF EXISTS sales_by_product_by_year_by_month''')
cur.execute('''CREATE TABLE sales_by_product_by_year_by_month (
    upc INTEGER, 
    description TEXT,
    department INTEGER,
    department_name TEXT,
    year INTEGER,
    month INTEGER,
    sales REAL,
    transactions REAL,
    items REAL)''')

with open("sales_by_product_by_year_by_month.csv",'r', encoding="utf-8") as ifile :
    next(ifile)
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO sales_by_product_by_year_by_month(upc,description,department,department_name,year,month,sales,transactions,items)
            VALUES (?,?,?,?,?,?,?,?,?)''', line)        
db.commit()

test = cur.execute('''SELECT * FROM sales_by_product_by_year_by_month LIMIT 10''')
print(list(test))

[('0.04DP9', 'GEN MERCH', 9, 'GEN MERCH', 2014, 7, 2.0, 1.0, 1.0), ('0.07DP14', 'JUICE BAR', 14, 'JUICE BAR', 2015, 10, 0.07, 1.0, 1.0), ('0.10DP3', 'BULK', 3, 'BULK', 2010, 10, 0.1, 1.0, 1.0), ('0.11DP4', 'REF GROCERY', 4, 'REF GROCERY', 2012, 11, 0.11, 1.0, 1.0), ('0.13DP12', 'HERBS&SPICES', 12, 'HERBS&SPICES', 2016, 2, 0.13, 1.0, 1.0), ('0.14DP1', 'PACKAGED GROCERY', 1, 'PACKAGED GROCERY', 2010, 6, 0.14, 1.0, 1.0), ('0.14DP1', 'PACKAGED GROCERY', 1, 'PACKAGED GROCERY', 2011, 8, 0.14, 1.0, 1.0), ('0.14DP9', 'GEN MERCH', 9, 'GEN MERCH', 2016, 3, 0.14, 1.0, 1.0), ('0.15DP9', 'GEN MERCH', 9, 'GEN MERCH', 2010, 12, 1.5, 1.0, 1.0), ('0.15DP9', 'GEN MERCH', 9, 'GEN MERCH', 2012, 1, 0.15, 1.0, 1.0)]


In [53]:
db.close()